In [1]:
import os
import itertools
import multiprocessing
from collections import namedtuple
from operator import itemgetter
import numpy as np
from tqdm import tqdm
from shutil import copyfile
from datetime import datetime
import pickle
from IPython.display import Javascript

save = '''
require(["base/js/namespace"],function(Jupyter) {
    Jupyter.notebook.save_checkpoint();
});
'''

In [2]:
INPUT_PATH = "data"
ROOT_OUTPUT_PATH = "solutions"
OUTPUT_PATH = os.path.join(ROOT_OUTPUT_PATH, "test")
os.makedirs(OUTPUT_PATH, exist_ok=True)
NB_NAME = 'hashcode2020_solution.ipynb'
SOLUTION_NAMES = pickle.loads(b'\x80\x04\x95\xc55\x00\x00\x00\x00\x00\x00]\x94(\x8c\x06people\x94\x8c\x07history\x94\x8c\x03way\x94\x8c\x03art\x94\x8c\x05world\x94\x8c\x0binformation\x94\x8c\x03map\x94\x8c\x03two\x94\x8c\x06family\x94\x8c\ngovernment\x94\x8c\x06health\x94\x8c\x06system\x94\x8c\x08computer\x94\x8c\x04meat\x94\x8c\x04year\x94\x8c\x06thanks\x94\x8c\x05music\x94\x8c\x06person\x94\x8c\x07reading\x94\x8c\x06method\x94\x8c\x04data\x94\x8c\x04food\x94\x8c\runderstanding\x94\x8c\x06theory\x94\x8c\x03law\x94\x8c\x04bird\x94\x8c\nliterature\x94\x8c\x07problem\x94\x8c\x08software\x94\x8c\x07control\x94\x8c\tknowledge\x94\x8c\x05power\x94\x8c\x07ability\x94\x8c\teconomics\x94\x8c\x04love\x94\x8c\x08internet\x94\x8c\ntelevision\x94\x8c\x07science\x94\x8c\x07library\x94\x8c\x06nature\x94\x8c\x04fact\x94\x8c\x07product\x94\x8c\x04idea\x94\x8c\x0btemperature\x94\x8c\ninvestment\x94\x8c\x04area\x94\x8c\x07society\x94\x8c\x08activity\x94\x8c\x05story\x94\x8c\x08industry\x94\x8c\x05media\x94\x8c\x05thing\x94\x8c\x04oven\x94\x8c\tcommunity\x94\x8c\ndefinition\x94\x8c\x06safety\x94\x8c\x07quality\x94\x8c\x0bdevelopment\x94\x8c\x08language\x94\x8c\nmanagement\x94\x8c\x06player\x94\x8c\x07variety\x94\x8c\x05video\x94\x8c\x04week\x94\x8c\x08security\x94\x8c\x07country\x94\x8c\x04exam\x94\x8c\x05movie\x94\x8c\x0corganization\x94\x8c\tequipment\x94\x8c\x07physics\x94\x8c\x08analysis\x94\x8c\x06policy\x94\x8c\x06series\x94\x8c\x07thought\x94\x8c\x05basis\x94\x8c\tboyfriend\x94\x8c\tdirection\x94\x8c\x08strategy\x94\x8c\ntechnology\x94\x8c\x04army\x94\x8c\x06camera\x94\x8c\x07freedom\x94\x8c\x05paper\x94\x8c\x0benvironment\x94\x8c\x05child\x94\x8c\x08instance\x94\x8c\x05month\x94\x8c\x05truth\x94\x8c\tmarketing\x94\x8c\nuniversity\x94\x8c\x07writing\x94\x8c\x07article\x94\x8c\ndepartment\x94\x8c\ndifference\x94\x8c\x04goal\x94\x8c\x04news\x94\x8c\x08audience\x94\x8c\x07fishing\x94\x8c\x06growth\x94\x8c\x06income\x94\x8c\x08marriage\x94\x8c\x04user\x94\x8c\x0bcombination\x94\x8c\x07failure\x94\x8c\x07meaning\x94\x8c\x08medicine\x94\x8c\nphilosophy\x94\x8c\x07teacher\x94\x8c\rcommunication\x94\x8c\x05night\x94\x8c\tchemistry\x94\x8c\x07disease\x94\x8c\x04disk\x94\x8c\x06energy\x94\x8c\x06nation\x94\x8c\x04road\x94\x8c\x04role\x94\x8c\x04soup\x94\x8c\x0badvertising\x94\x8c\x08location\x94\x8c\x07success\x94\x8c\x08addition\x94\x8c\tapartment\x94\x8c\teducation\x94\x8c\x04math\x94\x8c\x06moment\x94\x8c\x08painting\x94\x8c\x08politics\x94\x8c\tattention\x94\x8c\x08decision\x94\x8c\x05event\x94\x8c\x08property\x94\x8c\x08shopping\x94\x8c\x07student\x94\x8c\x04wood\x94\x8c\x0bcompetition\x94\x8c\x0cdistribution\x94\x8c\rentertainment\x94\x8c\x06office\x94\x8c\npopulation\x94\x8c\tpresident\x94\x8c\x04unit\x94\x8c\x08category\x94\x8c\tcigarette\x94\x8c\x07context\x94\x8c\x0cintroduction\x94\x8c\x0bopportunity\x94\x8c\x0bperformance\x94\x8c\x06driver\x94\x8c\x06flight\x94\x8c\x06length\x94\x8c\x08magazine\x94\x8c\tnewspaper\x94\x8c\x0crelationship\x94\x8c\x08teaching\x94\x8c\x04cell\x94\x8c\x06dealer\x94\x8c\x06debate\x94\x8c\x07finding\x94\x8c\x04lake\x94\x8c\x06member\x94\x8c\x07message\x94\x8c\x05phone\x94\x8c\x05scene\x94\x8c\nappearance\x94\x8c\x0bassociation\x94\x8c\x07concept\x94\x8c\x08customer\x94\x8c\x05death\x94\x8c\ndiscussion\x94\x8c\x07housing\x94\x8c\tinflation\x94\x8c\tinsurance\x94\x8c\x04mood\x94\x8c\x05woman\x94\x8c\x06advice\x94\x8c\x05blood\x94\x8c\x06effort\x94\x8c\nexpression\x94\x8c\nimportance\x94\x8c\x07opinion\x94\x8c\x07payment\x94\x8c\x07reality\x94\x8c\x0eresponsibility\x94\x8c\tsituation\x94\x8c\x05skill\x94\x8c\tstatement\x94\x8c\x06wealth\x94\x8c\x0bapplication\x94\x8c\x04city\x94\x8c\x06county\x94\x8c\x05depth\x94\x8c\x06estate\x94\x8c\nfoundation\x94\x8c\x0bgrandmother\x94\x8c\x05heart\x94\x8c\x0bperspective\x94\x8c\x05photo\x94\x8c\x06recipe\x94\x8c\x06studio\x94\x8c\x05topic\x94\x8c\ncollection\x94\x8c\ndepression\x94\x8c\x0bimagination\x94\x8c\x07passion\x94\x8c\npercentage\x94\x8c\x08resource\x94\x8c\x07setting\x94\x8c\x02ad\x94\x8c\x06agency\x94\x8c\x07college\x94\x8c\nconnection\x94\x8c\tcriticism\x94\x8c\x04debt\x94\x8c\x0bdescription\x94\x8c\x06memory\x94\x8c\x08patience\x94\x8c\tsecretary\x94\x8c\x08solution\x94\x8c\x0eadministration\x94\x8c\x06aspect\x94\x8c\x08attitude\x94\x8c\x08director\x94\x8c\x0bpersonality\x94\x8c\npsychology\x94\x8c\x0erecommendation\x94\x8c\x08response\x94\x8c\tselection\x94\x8c\x07storage\x94\x8c\x07version\x94\x8c\x07alcohol\x94\x8c\x08argument\x94\x8c\tcomplaint\x94\x8c\x08contract\x94\x8c\x08emphasis\x94\x8c\x07highway\x94\x8c\x04loss\x94\x8c\nmembership\x94\x8c\npossession\x94\x8c\x0bpreparation\x94\x8c\x05steak\x94\x8c\x05union\x94\x8c\tagreement\x94\x8c\x06cancer\x94\x8c\x08currency\x94\x8c\nemployment\x94\x8c\x0bengineering\x94\x8c\x05entry\x94\x8c\x0binteraction\x94\x8c\x05limit\x94\x8c\x07mixture\x94\x8c\npreference\x94\x8c\x06region\x94\x8c\x08republic\x94\x8c\x04seat\x94\x8c\ttradition\x94\x8c\x05virus\x94\x8c\x05actor\x94\x8c\tclassroom\x94\x8c\x08delivery\x94\x8c\x06device\x94\x8c\ndifficulty\x94\x8c\x05drama\x94\x8c\x08election\x94\x8c\x06engine\x94\x8c\x08football\x94\x8c\x08guidance\x94\x8c\x05hotel\x94\x8c\x05match\x94\x8c\x05owner\x94\x8c\x08priority\x94\x8c\nprotection\x94\x8c\nsuggestion\x94\x8c\x07tension\x94\x8c\tvariation\x94\x8c\x07anxiety\x94\x8c\natmosphere\x94\x8c\tawareness\x94\x8c\x05bread\x94\x8c\x07climate\x94\x8c\ncomparison\x94\x8c\tconfusion\x94\x8c\x0cconstruction\x94\x8c\x08elevator\x94\x8c\x07emotion\x94\x8c\x08employee\x94\x8c\x08employer\x94\x8c\x05guest\x94\x8c\x06height\x94\x8c\nleadership\x94\x8c\x04mall\x94\x8c\x07manager\x94\x8c\toperation\x94\x8c\trecording\x94\x8c\x07respect\x94\x8c\x06sample\x94\x8c\x0etransportation\x94\x8c\x06boring\x94\x8c\x07charity\x94\x8c\x06cousin\x94\x8c\x08disaster\x94\x8c\x06editor\x94\x8c\nefficiency\x94\x8c\nexcitement\x94\x8c\x06extent\x94\x8c\x08feedback\x94\x8c\x06guitar\x94\x8c\x08homework\x94\x8c\x06leader\x94\x8c\x03mom\x94\x8c\x07outcome\x94\x8c\npermission\x94\x8c\x0cpresentation\x94\x8c\tpromotion\x94\x8c\nreflection\x94\x8c\x0crefrigerator\x94\x8c\nresolution\x94\x8c\x07revenue\x94\x8c\x07session\x94\x8c\x06singer\x94\x8c\x06tennis\x94\x8c\x06basket\x94\x8c\x05bonus\x94\x8c\x07cabinet\x94\x8c\tchildhood\x94\x8c\x06church\x94\x8c\x07clothes\x94\x8c\x06coffee\x94\x8c\x06dinner\x94\x8c\x07drawing\x94\x8c\x04hair\x94\x8c\x07hearing\x94\x8c\ninitiative\x94\x8c\x08judgment\x94\x8c\x03lab\x94\x8c\x0bmeasurement\x94\x8c\x04mode\x94\x8c\x03mud\x94\x8c\x06orange\x94\x8c\x06poetry\x94\x8c\x06police\x94\x8c\x0bpossibility\x94\x8c\tprocedure\x94\x8c\x05queen\x94\x8c\x05ratio\x94\x8c\x08relation\x94\x8c\nrestaurant\x94\x8c\x0csatisfaction\x94\x8c\x06sector\x94\x8c\tsignature\x94\x8c\x0csignificance\x94\x8c\x04song\x94\x8c\x05tooth\x94\x8c\x04town\x94\x8c\x07vehicle\x94\x8c\x06volume\x94\x8c\x04wife\x94\x8c\x08accident\x94\x8c\x07airport\x94\x8c\x0bappointment\x94\x8c\x07arrival\x94\x8c\nassumption\x94\x8c\x08baseball\x94\x8c\x07chapter\x94\x8c\tcommittee\x94\x8c\x0cconversation\x94\x8c\x08database\x94\x8c\nenthusiasm\x94\x8c\x05error\x94\x8c\x0bexplanation\x94\x8c\x06farmer\x94\x8c\x04gate\x94\x8c\x04girl\x94\x8c\x04hall\x94\x8c\thistorian\x94\x8c\x08hospital\x94\x8c\x06injury\x94\x8c\x0binstruction\x94\x8c\x0bmaintenance\x94\x8c\x0cmanufacturer\x94\x8c\x04meal\x94\x8c\nperception\x94\x8c\x03pie\x94\x8c\x04poem\x94\x8c\x08presence\x94\x8c\x08proposal\x94\x8c\treception\x94\x8c\x0breplacement\x94\x8c\nrevolution\x94\x8c\x05river\x94\x8c\x03son\x94\x8c\x06speech\x94\x8c\x03tea\x94\x8c\x07village\x94\x8c\x07warning\x94\x8c\x06winner\x94\x8c\x06worker\x94\x8c\x06writer\x94\x8c\nassistance\x94\x8c\x06breath\x94\x8c\x05buyer\x94\x8c\x05chest\x94\x8c\tchocolate\x94\x8c\nconclusion\x94\x8c\x0ccontribution\x94\x8c\x06cookie\x94\x8c\x07courage\x94\x8c\x03dad\x94\x8c\x04desk\x94\x8c\x06drawer\x94\x8c\restablishment\x94\x8c\x0bexamination\x94\x8c\x07garbage\x94\x8c\x07grocery\x94\x8c\x05honey\x94\x8c\nimpression\x94\x8c\x0bimprovement\x94\x8c\x0cindependence\x94\x8c\x06insect\x94\x8c\ninspection\x94\x8c\tinspector\x94\x8c\x04king\x94\x8c\x06ladder\x94\x8c\x04menu\x94\x8c\x07penalty\x94\x8c\x05piano\x94\x8c\x06potato\x94\x8c\nprofession\x94\x8c\tprofessor\x94\x8c\x08quantity\x94\x8c\x08reaction\x94\x8c\x0brequirement\x94\x8c\x05salad\x94\x8c\x06sister\x94\x8c\x0bsupermarket\x94\x8c\x06tongue\x94\x8c\x08weakness\x94\x8c\x07wedding\x94\x8c\x06affair\x94\x8c\x08ambition\x94\x8c\x07analyst\x94\x8c\x05apple\x94\x8c\nassignment\x94\x8c\tassistant\x94\x8c\x08bathroom\x94\x8c\x07bedroom\x94\x8c\x04beer\x94\x8c\x08birthday\x94\x8c\x0bcelebration\x94\x8c\x0cchampionship\x94\x8c\x05cheek\x94\x8c\x06client\x94\x8c\x0bconsequence\x94\x8c\tdeparture\x94\x8c\x07diamond\x94\x8c\x04dirt\x94\x8c\x03ear\x94\x8c\x07fortune\x94\x8c\nfriendship\x94\x8c\x07funeral\x94\x8c\x04gene\x94\x8c\ngirlfriend\x94\x8c\x03hat\x94\x8c\nindication\x94\x8c\tintention\x94\x8c\x04lady\x94\x8c\x08midnight\x94\x8c\x0bnegotiation\x94\x8c\nobligation\x94\x8c\tpassenger\x94\x8c\x05pizza\x94\x8c\x08platform\x94\x8c\x04poet\x94\x8c\tpollution\x94\x8c\x0brecognition\x94\x8c\nreputation\x94\x8c\x05shirt\x94\x8c\x03sir\x94\x8c\x07speaker\x94\x8c\x08stranger\x94\x8c\x07surgery\x94\x8c\x08sympathy\x94\x8c\x04tale\x94\x8c\x06throat\x94\x8c\x07trainer\x94\x8c\x05uncle\x94\x8c\x05youth\x94\x8c\x04time\x94\x8c\x04work\x94\x8c\x04film\x94\x8c\x05water\x94\x8c\x05money\x94\x8c\x07example\x94\x8c\x05while\x94\x8c\x08business\x94\x8c\x05study\x94\x8c\x04game\x94\x8c\x04life\x94\x8c\x04form\x94\x8c\x03air\x94\x8c\x03day\x94\x8c\x05place\x94\x8c\x06number\x94\x8c\x04part\x94\x8c\x05field\x94\x8c\x04fish\x94\x8c\x04back\x94\x8c\x07process\x94\x8c\x04heat\x94\x8c\x04hand\x94\x8c\nexperience\x94\x8c\x03job\x94\x8c\x04book\x94\x8c\x03end\x94\x8c\x05point\x94\x8c\x04type\x94\x8c\x04home\x94\x8c\x07economy\x94\x8c\x05value\x94\x8c\x04body\x94\x8c\x06market\x94\x8c\x05guide\x94\x8c\x08interest\x94\x8c\x05state\x94\x8c\x05radio\x94\x8c\x06course\x94\x8c\x07company\x94\x8c\x05price\x94\x8c\x04size\x94\x8c\x04card\x94\x8c\x04list\x94\x8c\x04mind\x94\x8c\x05trade\x94\x8c\x04line\x94\x8c\x04care\x94\x8c\x05group\x94\x8c\x04risk\x94\x8c\x04word\x94\x8c\x03fat\x94\x8c\x05force\x94\x8c\x03key\x94\x8c\x05light\x94\x8c\x08training\x94\x8c\x04name\x94\x8c\x06school\x94\x8c\x03top\x94\x8c\x06amount\x94\x8c\x05level\x94\x8c\x05order\x94\x8c\x08practice\x94\x8c\x08research\x94\x8c\x05sense\x94\x8c\x07service\x94\x8c\x05piece\x94\x8c\x03web\x94\x8c\x04boss\x94\x8c\x05sport\x94\x8c\x03fun\x94\x8c\x05house\x94\x8c\x04page\x94\x8c\x04term\x94\x8c\x04test\x94\x8c\x06answer\x94\x8c\x05sound\x94\x8c\x05focus\x94\x8c\x06matter\x94\x8c\x04kind\x94\x8c\x04soil\x94\x8c\x05board\x94\x8c\x03oil\x94\x8c\x07picture\x94\x8c\x06access\x94\x8c\x06garden\x94\x8c\x05range\x94\x8c\x04rate\x94\x8c\x06reason\x94\x8c\x06future\x94\x8c\x04site\x94\x8c\x06demand\x94\x8c\x08exercise\x94\x8c\x05image\x94\x8c\x04case\x94\x8c\x05cause\x94\x8c\x05coast\x94\x8c\x06action\x94\x8c\x03age\x94\x8c\x03bad\x94\x8c\x04boat\x94\x8c\x06record\x94\x8c\x06result\x94\x8c\x07section\x94\x8c\x08building\x94\x8c\x05mouse\x94\x8c\x04cash\x94\x8c\x05class\x94\x8c\x07nothing\x94\x8c\x06period\x94\x8c\x04plan\x94\x8c\x05store\x94\x8c\x03tax\x94\x8c\x04side\x94\x8c\x07subject\x94\x8c\x05space\x94\x8c\x04rule\x94\x8c\x05stock\x94\x8c\x07weather\x94\x8c\x06chance\x94\x8c\x06figure\x94\x8c\x03man\x94\x8c\x05model\x94\x8c\x06source\x94\x8c\tbeginning\x94\x8c\x05earth\x94\x8c\x07program\x94\x8c\x07chicken\x94\x8c\x06design\x94\x8c\x07feature\x94\x8c\x04head\x94\x8c\x08material\x94\x8c\x07purpose\x94\x8c\x08question\x94\x8c\x04rock\x94\x8c\x04salt\x94\x8c\x03act\x94\x8c\x05birth\x94\x8c\x03car\x94\x8c\x03dog\x94\x8c\x06object\x94\x8c\x05scale\x94\x8c\x03sun\x94\x8c\x04note\x94\x8c\x06profit\x94\x8c\x04rent\x94\x8c\x05speed\x94\x8c\x05style\x94\x8c\x03war\x94\x8c\x04bank\x94\x8c\x05craft\x94\x8c\x04half\x94\x8c\x06inside\x94\x8c\x07outside\x94\x8c\x08standard\x94\x8c\x03bus\x94\x8c\x08exchange\x94\x8c\x03eye\x94\x8c\x04fire\x94\x8c\x08position\x94\x8c\x08pressure\x94\x8c\x06stress\x94\x8c\tadvantage\x94\x8c\x07benefit\x94\x8c\x03box\x94\x8c\x05frame\x94\x8c\x05issue\x94\x8c\x04step\x94\x8c\x05cycle\x94\x8c\x04face\x94\x8c\x04item\x94\x8c\x05metal\x94\x8c\x05paint\x94\x8c\x06review\x94\x8c\x04room\x94\x8c\x06screen\x94\x8c\tstructure\x94\x8c\x04view\x94\x8c\x07account\x94\x8c\x04ball\x94\x8c\ndiscipline\x94\x8c\x06medium\x94\x8c\x05share\x94\x8c\x07balance\x94\x8c\x03bit\x94\x8c\x05black\x94\x8c\x06bottom\x94\x8c\x06choice\x94\x8c\x04gift\x94\x8c\x06impact\x94\x8c\x07machine\x94\x8c\x05shape\x94\x8c\x04tool\x94\x8c\x04wind\x94\x8c\x07address\x94\x8c\x07average\x94\x8c\x06career\x94\x8c\x07culture\x94\x8c\x07morning\x94\x8c\x03pot\x94\x8c\x04sign\x94\x8c\x05table\x94\x8c\x04task\x94\x8c\tcondition\x94\x8c\x07contact\x94\x8c\x06credit\x94\x8c\x03egg\x94\x8c\x04hope\x94\x8c\x03ice\x94\x8c\x07network\x94\x8c\x05north\x94\x8c\x06square\x94\x8c\x07attempt\x94\x8c\x04date\x94\x8c\x06effect\x94\x8c\x04link\x94\x8c\x04post\x94\x8c\x04star\x94\x8c\x05voice\x94\x8c\x07capital\x94\x8c\tchallenge\x94\x8c\x06friend\x94\x8c\x04self\x94\x8c\x04shot\x94\x8c\x05brush\x94\x8c\x06couple\x94\x8c\x04exit\x94\x8c\x05front\x94\x8c\x08function\x94\x8c\x04lack\x94\x8c\x06living\x94\x8c\x05plant\x94\x8c\x07plastic\x94\x8c\x04spot\x94\x8c\x06summer\x94\x8c\x05taste\x94\x8c\x05theme\x94\x8c\x05track\x94\x8c\x04wing\x94\x8c\x05brain\x94\x8c\x06button\x94\x8c\x05click\x94\x8c\x06desire\x94\x8c\x04foot\x94\x8c\x03gas\x94\x8c\tinfluence\x94\x8c\x06notice\x94\x8c\x04rain\x94\x8c\x04wall\x94\x8c\x04base\x94\x8c\x06damage\x94\x8c\x08distance\x94\x8c\x07feeling\x94\x8c\x04pair\x94\x8c\x07savings\x94\x8c\x05staff\x94\x8c\x05sugar\x94\x8c\x06target\x94\x8c\x04text\x94\x8c\x06animal\x94\x8c\x06author\x94\x8c\x06budget\x94\x8c\x08discount\x94\x8c\x04file\x94\x8c\x06ground\x94\x8c\x06lesson\x94\x8c\x06minute\x94\x8c\x07officer\x94\x8c\x05phase\x94\x8c\treference\x94\x8c\x08register\x94\x8c\x03sky\x94\x8c\x05stage\x94\x8c\x05stick\x94\x8c\x05title\x94\x8c\x07trouble\x94\x8c\x04bowl\x94\x8c\x06bridge\x94\x8c\x08campaign\x94\x8c\tcharacter\x94\x8c\x04club\x94\x8c\x04edge\x94\x8c\x08evidence\x94\x8c\x03fan\x94\x8c\x06letter\x94\x8c\x04lock\x94\x8c\x07maximum\x94\x8c\x05novel\x94\x8c\x06option\x94\x8c\x04pack\x94\x8c\x04park\x94\x8c\x06plenty\x94\x8c\x07quarter\x94\x8c\x04skin\x94\x8c\x04sort\x94\x8c\x06weight\x94\x8c\x04baby\x94\x8c\nbackground\x94\x8c\x05carry\x94\x8c\x04dish\x94\x8c\x06factor\x94\x8c\x05fruit\x94\x8c\x05glass\x94\x8c\x05joint\x94\x8c\x06master\x94\x8c\x06muscle\x94\x8c\x03red\x94\x8c\x08strength\x94\x8c\x07traffic\x94\x8c\x04trip\x94\x8c\tvegetable\x94\x8c\x06appeal\x94\x8c\x05chart\x94\x8c\x04gear\x94\x8c\x05ideal\x94\x8c\x07kitchen\x94\x8c\x04land\x94\x8c\x03log\x94\x8c\x06mother\x94\x8c\x03net\x94\x8c\x05party\x94\x8c\tprinciple\x94\x8c\x08relative\x94\x8c\x04sale\x94\x8c\x06season\x94\x8c\x06signal\x94\x8c\x06spirit\x94\x8c\x06street\x94\x8c\x04tree\x94\x8c\x04wave\x94\x8c\x04belt\x94\x8c\x05bench\x94\x8c\ncommission\x94\x8c\x04copy\x94\x8c\x04drop\x94\x8c\x07minimum\x94\x8c\x04path\x94\x8c\x08progress\x94\x8c\x07project\x94\x8c\x03sea\x94\x8c\x05south\x94\x8c\x06status\x94\x8c\x05stuff\x94\x8c\x06ticket\x94\x8c\x04tour\x94\x8c\x05angle\x94\x8c\x04blue\x94\x8c\tbreakfast\x94\x8c\nconfidence\x94\x8c\x08daughter\x94\x8c\x06degree\x94\x8c\x06doctor\x94\x8c\x03dot\x94\x8c\x05dream\x94\x8c\x04duty\x94\x8c\x05essay\x94\x8c\x06father\x94\x8c\x03fee\x94\x8c\x07finance\x94\x8c\x04hour\x94\x8c\x05juice\x94\x8c\x04luck\x94\x8c\x04milk\x94\x8c\x05mouth\x94\x8c\x05peace\x94\x8c\x04pipe\x94\x8c\x06stable\x94\x8c\x05storm\x94\x8c\tsubstance\x94\x8c\x04team\x94\x8c\x05trick\x94\x8c\tafternoon\x94\x8c\x03bat\x94\x8c\x05beach\x94\x8c\x05blank\x94\x8c\x05catch\x94\x8c\x05chain\x94\x8c\rconsideration\x94\x8c\x05cream\x94\x8c\x04crew\x94\x8c\x06detail\x94\x8c\x04gold\x94\x8c\tinterview\x94\x8c\x03kid\x94\x8c\x04mark\x94\x8c\x07mission\x94\x8c\x04pain\x94\x8c\x08pleasure\x94\x8c\x05score\x94\x8c\x05screw\x94\x8c\x03sex\x94\x8c\x04shop\x94\x8c\x06shower\x94\x8c\x04suit\x94\x8c\x04tone\x94\x8c\x06window\x94\x8c\x05agent\x94\x8c\x04band\x94\x8c\x04bath\x94\x8c\x05block\x94\x8c\x04bone\x94\x8c\x08calendar\x94\x8c\tcandidate\x94\x8c\x03cap\x94\x8c\x04coat\x94\x8c\x07contest\x94\x8c\x06corner\x94\x8c\x05court\x94\x8c\x03cup\x94\x8c\x08district\x94\x8c\x04door\x94\x8c\x04east\x94\x8c\x06finger\x94\x8c\x06garage\x94\x8c\tguarantee\x94\x8c\x04hole\x94\x8c\x04hook\x94\x8c\timplement\x94\x8c\x05layer\x94\x8c\x07lecture\x94\x8c\x03lie\x94\x8c\x06manner\x94\x8c\x07meeting\x94\x8c\x04nose\x94\x8c\x07parking\x94\x8c\x07partner\x94\x8c\x07profile\x94\x8c\x04rice\x94\x8c\x07routine\x94\x8c\x08schedule\x94\x8c\x08swimming\x94\x8c\ttelephone\x94\x8c\x03tip\x94\x8c\x06winter\x94\x8c\x07airline\x94\x8c\x03bag\x94\x8c\x06battle\x94\x8c\x03bed\x94\x8c\x04bill\x94\x8c\x06bother\x94\x8c\x04cake\x94\x8c\x04code\x94\x8c\x05curve\x94\x8c\x08designer\x94\x8c\tdimension\x94\x8c\x05dress\x94\x8c\x04ease\x94\x8c\temergency\x94\x8c\x07evening\x94\x8c\textension\x94\x8c\x04farm\x94\x8c\x05fight\x94\x8c\x03gap\x94\x8c\x05grade\x94\x8c\x07holiday\x94\x8c\x06horror\x94\x8c\x05horse\x94\x8c\x04host\x94\x8c\x07husband\x94\x8c\x04loan\x94\x8c\x07mistake\x94\x8c\x08mountain\x94\x8c\x04nail\x94\x8c\x05noise\x94\x8c\x08occasion\x94\x8c\x07package\x94\x8c\x07patient\x94\x8c\x05pause\x94\x8c\x06phrase\x94\x8c\x05proof\x94\x8c\x04race\x94\x8c\x06relief\x94\x8c\x04sand\x94\x8c\x08sentence\x94\x8c\x08shoulder\x94\x8c\x05smoke\x94\x8c\x07stomach\x94\x8c\x06string\x94\x8c\x07tourist\x94\x8c\x05towel\x94\x8c\x08vacation\x94\x8c\x04west\x94\x8c\x05wheel\x94\x8c\x04wine\x94\x8c\x03arm\x94\x8c\x05aside\x94\x8c\tassociate\x94\x8c\x03bet\x94\x8c\x04blow\x94\x8c\x06border\x94\x8c\x06branch\x94\x8c\x06breast\x94\x8c\x07brother\x94\x8c\x05buddy\x94\x8c\x05bunch\x94\x8c\x04chip\x94\x8c\x05coach\x94\x8c\x05cross\x94\x8c\x08document\x94\x8c\x05draft\x94\x8c\x04dust\x94\x8c\x06expert\x94\x8c\x05floor\x94\x8c\x03god\x94\x8c\x04golf\x94\x8c\x05habit\x94\x8c\x04iron\x94\x8c\x05judge\x94\x8c\x05knife\x94\x8c\tlandscape\x94\x8c\x06league\x94\x8c\x04mail\x94e(\x8c\x04mess\x94\x8c\x06native\x94\x8c\x07opening\x94\x8c\x06parent\x94\x8c\x07pattern\x94\x8c\x03pin\x94\x8c\x04pool\x94\x8c\x05pound\x94\x8c\x07request\x94\x8c\x06salary\x94\x8c\x05shame\x94\x8c\x07shelter\x94\x8c\x04shoe\x94\x8c\x06silver\x94\x8c\x06tackle\x94\x8c\x04tank\x94\x8c\x05trust\x94\x8c\x06assist\x94\x8c\x04bake\x94\x8c\x03bar\x94\x8c\x04bell\x94\x8c\x04bike\x94\x8c\x05blame\x94\x8c\x03boy\x94\x8c\x05brick\x94\x8c\x05chair\x94\x8c\x06closet\x94\x8c\x04clue\x94\x8c\x06collar\x94\x8c\x07comment\x94\x8c\nconference\x94\x8c\x05devil\x94\x8c\x04diet\x94\x8c\x04fear\x94\x8c\x04fuel\x94\x8c\x05glove\x94\x8c\x06jacket\x94\x8c\x05lunch\x94\x8c\x07monitor\x94\x8c\x08mortgage\x94\x8c\x05nurse\x94\x8c\x04pace\x94\x8c\x05panic\x94\x8c\x04peak\x94\x8c\x05plane\x94\x8c\x06reward\x94\x8c\x03row\x94\x8c\x08sandwich\x94\x8c\x05shock\x94\x8c\x05spite\x94\x8c\x05spray\x94\x8c\x08surprise\x94\x8c\x04till\x94\x8c\ntransition\x94\x8c\x07weekend\x94\x8c\x07welcome\x94\x8c\x04yard\x94\x8c\x05alarm\x94\x8c\x04bend\x94\x8c\x07bicycle\x94\x8c\x04bite\x94\x8c\x05blind\x94\x8c\x06bottle\x94\x8c\x05cable\x94\x8c\x06candle\x94\x8c\x05clerk\x94\x8c\x05cloud\x94\x8c\x07concert\x94\x8c\x07counter\x94\x8c\x06flower\x94\x8c\x0bgrandfather\x94\x8c\x04harm\x94\x8c\x04knee\x94\x8c\x06lawyer\x94\x8c\x07leather\x94\x8c\x04load\x94\x8c\x06mirror\x94\x8c\x04neck\x94\x8c\x07pension\x94\x8c\x05plate\x94\x8c\x06purple\x94\x8c\x04ruin\x94\x8c\x04ship\x94\x8c\x05skirt\x94\x8c\x05slice\x94\x8c\x04snow\x94\x8c\nspecialist\x94\x8c\x06stroke\x94\x8c\x06switch\x94\x8c\x05trash\x94\x8c\x04tune\x94\x8c\x04zone\x94\x8c\x05anger\x94\x8c\x05award\x94\x8c\x03bid\x94\x8c\x06bitter\x94\x8c\x04boot\x94\x8c\x03bug\x94\x8c\x04camp\x94\x8c\x05candy\x94\x8c\x06carpet\x94\x8c\x03cat\x94\x8c\x08champion\x94\x8c\x07channel\x94\x8c\x05clock\x94\x8c\x07comfort\x94\x8c\x03cow\x94\x8c\x05crack\x94\x8c\x08engineer\x94\x8c\x08entrance\x94\x8c\x05fault\x94\x8c\x05grass\x94\x8c\x03guy\x94\x8c\x04hell\x94\x8c\thighlight\x94\x8c\x08incident\x94\x8c\x06island\x94\x8c\x04joke\x94\x8c\x04jury\x94\x8c\x03leg\x94\x8c\x03lip\x94\x8c\x04mate\x94\x8c\x05motor\x94\x8c\x05nerve\x94\x8c\x07passage\x94\x8c\x03pen\x94\x8c\x05pride\x94\x8c\x06priest\x94\x8c\x05prize\x94\x8c\x07promise\x94\x8c\x08resident\x94\x8c\x06resort\x94\x8c\x04ring\x94\x8c\x04roof\x94\x8c\x04rope\x94\x8c\x04sail\x94\x8c\x06scheme\x94\x8c\x06script\x94\x8c\x04sock\x94\x8c\x07station\x94\x8c\x03toe\x94\x8c\x05tower\x94\x8c\x05truck\x94\x8c\x07witness\x94\x8c\x01a\x94\x8c\x03you\x94\x8c\x02it\x94\x8c\x03can\x94\x8c\x04will\x94\x8c\x02if\x94\x8c\x03one\x94\x8c\x04many\x94\x8c\x04most\x94\x8c\x05other\x94\x8c\x03use\x94\x8c\x04make\x94\x8c\x04good\x94\x8c\x04look\x94\x8c\x04help\x94\x8c\x02go\x94\x8c\x05great\x94\x8c\x05being\x94\x8c\x03few\x94\x8c\x05might\x94\x8c\x05still\x94\x8c\x06public\x94\x8c\x04read\x94\x8c\x04keep\x94\x8c\x05start\x94\x8c\x04give\x94\x8c\x05human\x94\x8c\x05local\x94\x8c\x07general\x94\x8c\x03she\x94\x8c\x08specific\x94\x8c\x04long\x94\x8c\x04play\x94\x8c\x04feel\x94\x8c\x04high\x94\x8c\x07tonight\x94\x8c\x03put\x94\x8c\x06common\x94\x8c\x03set\x94\x8c\x06change\x94\x8c\x06simple\x94\x8c\x04past\x94\x8c\x03big\x94\x8c\x08possible\x94\x8c\nparticular\x94\x8c\x05today\x94\x8c\x05major\x94\x8c\x08personal\x94\x8c\x07current\x94\x8c\x08national\x94\x8c\x03cut\x94\x8c\x07natural\x94\x8c\x08physical\x94\x8c\x04show\x94\x8c\x03try\x94\x8c\x05check\x94\x8c\x06second\x94\x8c\x04call\x94\x8c\x04move\x94\x8c\x03pay\x94\x8c\x03let\x94\x8c\x08increase\x94\x8c\x06single\x94\x8c\nindividual\x94\x8c\x04turn\x94\x8c\x03ask\x94\x8c\x03buy\x94\x8c\x05guard\x94\x8c\x04hold\x94\x8c\x04main\x94\x8c\x05offer\x94\x8c\tpotential\x94\x8c\x0cprofessional\x94\x8c\rinternational\x94\x8c\x06travel\x94\x8c\x04cook\x94\x8c\x0balternative\x94\x8c\tfollowing\x94\x8c\x07special\x94\x8c\x07working\x94\x8c\x05whole\x94\x8c\x05dance\x94\x8c\x06excuse\x94\x8c\x04cold\x94\x8c\ncommercial\x94\x8c\x03low\x94\x8c\x08purchase\x94\x8c\x04deal\x94\x8c\x07primary\x94\x8c\x05worth\x94\x8c\x04fall\x94\x8c\tnecessary\x94\x8c\x08positive\x94\x8c\x07produce\x94\x8c\x06search\x94\x8c\x07present\x94\x8c\x05spend\x94\x8c\x04talk\x94\x8c\x08creative\x94\x8c\x04tell\x94\x8c\x04cost\x94\x8c\x05drive\x94\x8c\x05green\x94\x8c\x07support\x94\x8c\x04glad\x94\x8c\x06remove\x94\x8c\x06return\x94\x8c\x03run\x94\x8c\x07complex\x94\x8c\x03due\x94\x8c\teffective\x94\x8c\x06middle\x94\x8c\x07regular\x94\x8c\x07reserve\x94\x8c\x0bindependent\x94\x8c\x05leave\x94\x8c\x08original\x94\x8c\x05reach\x94\x8c\x04rest\x94\x8c\x05serve\x94\x8c\x05watch\x94\x8c\tbeautiful\x94\x8c\x06charge\x94\x8c\x06active\x94\x8c\x05break\x94\x8c\x08negative\x94\x8c\x04safe\x94\x8c\x04stay\x94\x8c\x05visit\x94\x8c\x06visual\x94\x8c\x06affect\x94\x8c\x05cover\x94\x8c\x06report\x94\x8c\x04rise\x94\x8c\x04walk\x94\x8c\x05white\x94\x8c\x06beyond\x94\x8c\x06junior\x94\x8c\x04pick\x94\x8c\x06unique\x94\x8c\x08anything\x94\x8c\x07classic\x94\x8c\x05final\x94\x8c\x04lift\x94\x8c\x03mix\x94\x8c\x07private\x94\x8c\x04stop\x94\x8c\x05teach\x94\x8c\x07western\x94\x8c\x07concern\x94\x8c\x08familiar\x94\x8c\x03fly\x94\x8c\x08official\x94\x8c\x05broad\x94\x8c\x0bcomfortable\x94\x8c\x04gain\x94\x8c\x05maybe\x94\x8c\x04rich\x94\x8c\x04save\x94\x8c\x05stand\x94\x8c\x05young\x94\x8c\x05heavy\x94\x8c\x05hello\x94\x8c\x04lead\x94\x8c\x06listen\x94\x8c\x08valuable\x94\x8c\x05worry\x94\x8c\x06handle\x94\x8c\x07leading\x94\x8c\x04meet\x94\x8c\x07release\x94\x8c\x04sell\x94\x8c\x06finish\x94\x8c\x06normal\x94\x8c\x05press\x94\x8c\x04ride\x94\x8c\x06secret\x94\x8c\x06spread\x94\x8c\x06spring\x94\x8c\x05tough\x94\x8c\x04wait\x94\x8c\x05brown\x94\x8c\x04deep\x94\x8c\x07display\x94\x8c\x04flow\x94\x8c\x03hit\x94\x8c\tobjective\x94\x8c\x05shoot\x94\x8c\x05touch\x94\x8c\x06cancel\x94\x8c\x08chemical\x94\x8c\x03cry\x94\x8c\x04dump\x94\x8c\x07extreme\x94\x8c\x04push\x94\x8c\x08conflict\x94\x8c\x03eat\x94\x8c\x04fill\x94\x8c\x06formal\x94\x8c\x04jump\x94\x8c\x04kick\x94\x8c\x08opposite\x94\x8c\x04pass\x94\x8c\x05pitch\x94\x8c\x06remote\x94\x8c\x05total\x94\x8c\x05treat\x94\x8c\x04vast\x94\x8c\x05abuse\x94\x8c\x04beat\x94\x8c\x04burn\x94\x8c\x07deposit\x94\x8c\x05print\x94\x8c\x05raise\x94\x8c\x05sleep\x94\x8c\tsomewhere\x94\x8c\x07advance\x94\x8c\x08anywhere\x94\x8c\x07consist\x94\x8c\x04dark\x94\x8c\x06double\x94\x8c\x04draw\x94\x8c\x05equal\x94\x8c\x03fix\x94\x8c\x04hire\x94\x8c\x08internal\x94\x8c\x04join\x94\x8c\x04kill\x94\x8c\tsensitive\x94\x8c\x03tap\x94\x8c\x03win\x94\x8c\x06attack\x94\x8c\x05claim\x94\x8c\x08constant\x94\x8c\x04drag\x94\x8c\x05drink\x94\x8c\x05guess\x94\x8c\x05minor\x94\x8c\x04pull\x94\x8c\x03raw\x94\x8c\x04soft\x94\x8c\x05solid\x94\x8c\x04wear\x94\x8c\x05weird\x94\x8c\x06wonder\x94\x8c\x06annual\x94\x8c\x05count\x94\x8c\x04dead\x94\x8c\x05doubt\x94\x8c\x04feed\x94\x8c\x07forever\x94\x8c\x07impress\x94\x8c\x06nobody\x94\x8c\x06repeat\x94\x8c\x05round\x94\x8c\x04sing\x94\x8c\x05slide\x94\x8c\x05strip\x94\x8c\x07whereas\x94\x8c\x04wish\x94\x8c\x07combine\x94\x8c\x07command\x94\x8c\x03dig\x94\x8c\x06divide\x94\x8c\nequivalent\x94\x8c\x04hang\x94\x8c\x04hunt\x94\x8c\x07initial\x94\x8c\x05march\x94\x8c\x07mention\x94\x8c\tspiritual\x94\x8c\x06survey\x94\x8c\x03tie\x94\x8c\x05adult\x94\x8c\x05brief\x94\x8c\x05crazy\x94\x8c\x06escape\x94\x8c\x06gather\x94\x8c\x04hate\x94\x8c\x05prior\x94\x8c\x06repair\x94\x8c\x05rough\x94\x8c\x03sad\x94\x8c\x07scratch\x94\x8c\x04sick\x94\x8c\x06strike\x94\x8c\x06employ\x94\x8c\x08external\x94\x8c\x04hurt\x94\x8c\x07illegal\x94\x8c\x05laugh\x94\x8c\x03lay\x94\x8c\x06mobile\x94\x8c\x05nasty\x94\x8c\x08ordinary\x94\x8c\x07respond\x94\x8c\x05royal\x94\x8c\x06senior\x94\x8c\x05split\x94\x8c\x06strain\x94\x8c\x08struggle\x94\x8c\x04swim\x94\x8c\x05train\x94\x8c\x05upper\x94\x8c\x04wash\x94\x8c\x06yellow\x94\x8c\x07convert\x94\x8c\x05crash\x94\x8c\tdependent\x94\x8c\x04fold\x94\x8c\x05funny\x94\x8c\x04grab\x94\x8c\x04hide\x94\x8c\x04miss\x94\x8c\x06permit\x94\x8c\x05quote\x94\x8c\x07recover\x94\x8c\x07resolve\x94\x8c\x04roll\x94\x8c\x04sink\x94\x8c\x04slip\x94\x8c\x05spare\x94\x8c\x07suspect\x94\x8c\x05sweet\x94\x8c\x05swing\x94\x8c\x05twist\x94\x8c\x08upstairs\x94\x8c\x05usual\x94\x8c\x06abroad\x94\x8c\x05brave\x94\x8c\x04calm\x94\x8c\x0bconcentrate\x94\x8c\x08estimate\x94\x8c\x05grand\x94\x8c\x04male\x94\x8c\x04mine\x94\x8c\x06prompt\x94\x8c\x05quiet\x94\x8c\x06refuse\x94\x8c\x06regret\x94\x8c\x06reveal\x94\x8c\x04rush\x94\x8c\x05shake\x94\x8c\x05shift\x94\x8c\x05shine\x94\x8c\x05steal\x94\x8c\x04suck\x94\x8c\x08surround\x94\x8c\x07anybody\x94\x8c\x04bear\x94\x8c\tbrilliant\x94\x8c\x04dare\x94\x8c\x04dear\x94\x8c\x05delay\x94\x8c\x05drunk\x94\x8c\x06female\x94\x8c\x05hurry\x94\x8c\ninevitable\x94\x8c\x06invite\x94\x8c\x04kiss\x94\x8c\x04neat\x94\x8c\x03pop\x94\x8c\x05punch\x94\x8c\x04quit\x94\x8c\x05reply\x94\x8c\x0erepresentative\x94\x8c\x06resist\x94\x8c\x03rip\x94\x8c\x03rub\x94\x8c\x05silly\x94\x8c\x05smile\x94\x8c\x05spell\x94\x8c\x07stretch\x94\x8c\x06stupid\x94\x8c\x04tear\x94\x8c\ttemporary\x94\x8c\x08tomorrow\x94\x8c\x04wake\x94\x8c\x04wrap\x94\x8c\tyesterday\x94e.')

Solution runner and writer.

In [3]:
def mk_output_dir():
    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    simple_name = SOLUTION_NAMES[len(os.listdir(ROOT_OUTPUT_PATH))]
    OUTPUT_PATH = os.path.join(ROOT_OUTPUT_PATH, f"{current_time}_{simple_name}_{abs(hash(datetime.now()))}")
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    return OUTPUT_PATH

def write_bresult(s):
    with open(os.path.join(OUTPUT_PATH, "res.pkl"), "wb") as f:
        pickle.dump(s, f)

def load_bresult(s):
    with open(os.path.join(ROOT_OUTPUT_PATH, s, "res.pkl"), "rb") as f:
        return pickle.load(f)

def run(i):
    parsed_input = parse_input(os.path.join(INPUT_PATH, i))
    s = compute(parsed_input)
    score = total_score(parsed_input, s)
    print(f"%s: \t\t{score:,}"%(i.split("_")[0]))
    with open(os.path.join(OUTPUT_PATH, f"{i}_result"), "w") as f:
        f.writelines(map(lambda x: f"{x}\n", format_output(s)))
    return s, score

def new_sol():
    global OUTPUT_PATH
    OUTPUT_PATH = mk_output_dir()
    files = sorted([x for x in os.listdir(INPUT_PATH) if not x.startswith(".")])
    print(files)
    pool = multiprocessing.Pool(processes=len(files))
    res = pool.map(run, files)
    s, scores = list(map(itemgetter(0), res)), list(map(itemgetter(1), res))
    print(f"Total: \t\t{sum(scores):,}")
    pool.close()
    write_bresult(s)
    Javascript(save)
    copyfile(os.path.join(os.getcwd(), NB_NAME), os.path.join(OUTPUT_PATH, NB_NAME))
    return s

---

# EDIT FROM HERE ON

Data structures

In [4]:
Scanning = namedtuple('Scanning', 'num_books, num_libraries, days, scores, libraries')
Library = namedtuple('Library', 'id, num_books, signup_days, books_per_day, books')

EnhancedLibrary = namedtuple('EnhancedLibrary', 'id, num_books, signup_days, books_per_day, books, ' 
                             + 'unique_books, sum_scores, sum_unique_scores, max_output')

ScanTaskSignUp = namedtuple('ScanTaskSignUp', 'id, num_books, books')

Input parsing and output formatting (not writing)

In [5]:
def parse_input(fname, cache=True):
    if not cache:
        with open(fname) as f:
            data = f.read().splitlines()
            data = list(map(lambda x: list(map(int, x.split())), data))
            i = 2
            libid = 0
            libraries = []
            while i < len(data) - 1:
                libraries.append(Library(libid, data[i][0], data[i][1], data[i][2], data[i+1]))
                i += 2
                libid += 1

            return Scanning(data[0][0], data[0][1], data[0][2], data[1], libraries)
    else:
        # load from enhanced lib cache
        return ENHN_LIB[fname.split('/')[-1]]

def format_output(s):
    result = []
    result.append(f"{len(s)}")
    for x in s:
        result.append(f"{x.id} {len(x.books)}")
        result.append(" ".join(map(str, x.books)))
    return result

def enhance_libraries(scanning):
    D = scanning.days
    scores = scanning.scores

    for i, library in enumerate(scanning.libraries):
        max_output = (D - library.signup_days) * library.books_per_day
        sum_scores = sum([scores[x] for x in library.books])

        unique_books = set(library.books)
        for j, otherlibrary in enumerate(scanning.libraries):
            if(j != i):
                unique_books = unique_books - set(otherlibrary.books)
                
        sum_unique_scores = sum([scores[x] for x in unique_books])

        scanning.libraries[i] = EnhancedLibrary(library.id, library.num_books, library.signup_days, 
                                                library.books_per_day, library.books, 
                                                unique_books, sum_scores, sum_unique_scores, max_output)
        
    return scanning

Load scannings, enhance and store. This allows to later load the serialized enhanced libraries.

In [6]:
# Build enhanced libs
ENHN_FILE = 'enhanced_libs.pkl'
def read_and_enhance(i):
    parsed_input = parse_input(os.path.join(INPUT_PATH, i), cache=False)
    return enhance_libraries(parsed_input)

if not os.path.isfile(ENHN_FILE):
    files = sorted([x for x in os.listdir(INPUT_PATH) if not x.startswith(".")])
    pool = multiprocessing.Pool(processes=len(files))
    enhanced_libs = pool.map(read_and_enhance, files)
    pool.close()

# Load enhanced libs
with open(ENHN_FILE, 'rb') as f:
    ENHN_LIB = pickle.load(f)

Scoring function

In [7]:
def total_score(scanning, signups):
    days = scanning.days
    scores = scanning.scores
    
    sent_books = set()
    total = 0
    start_signup = 0
    
    for library in signups:
        signup_days = scanning.libraries[library.id].signup_days
        books_per_day = scanning.libraries[library.id].books_per_day
        books_to_send = library.books
        
        total_days = days - (signup_days+1) - start_signup
        start_signup += signup_days
        
        while((total_days >= 0) & (len(books_to_send)>0)):
            for book in books_to_send[0:books_per_day]:
                if(book not in sent_books):
                    total += scores[book]
                    sent_books = sent_books.union(set([book]))
            total_days -= 1
            books_to_send = books_to_send[(books_per_day)::]
    return total

Sorting helpers

In [8]:
def sort_libraries_by_field(images, field='signup_days', reverse=True):
    return sorted(images, key=itemgetter(Library._fields.index(field)), reverse=reverse)

def sort_enhlibraries_by_field(images, field='signup_days', reverse=True):
    return sorted(images, key=itemgetter(EnhancedLibrary._fields.index(field)), reverse=reverse)

def sort_books_by_scores_desc(global_scores, books):
    return [books[x] for x in sorted(range(len(books)),key=lambda x: global_scores[x], reverse=True)]

Prune duplicate books from signing up

In [9]:
def prune_dups(scanning, signup):
    libraries = scanning.libraries

    ids = list(map(itemgetter(ScanTaskSignUp._fields.index('id')), signup))

    books_per_day = [libraries[i].books_per_day for i in ids]
    signup_days = list(np.cumsum([libraries[i].signup_days for i in ids]))
    id_to_task = dict((x.id, x.books) for x in signup)
    

    all_seen = set()
    for t in range(scanning.days):
        modified = True
        while modified:
            modified = False

            cur_sent = set()
            for l in ids:
                if t >= signup_days[l]:
                    books_of_cur_l = id_to_task[l]
                    for j in range(books_per_day[l]):
                        if (t - signup_days[l]) * books_per_day[l] + j < len(books_of_cur_l):
                            cur_b = books_of_cur_l[(t - signup_days[l]) * books_per_day[l] + j]
                            if cur_b not in all_seen:
                                all_seen.add(cur_b)
                            else:
                                del books_of_cur_l[(t - signup_days[l]) * books_per_day[l] + j]
                                modified = True
                                continue
    signup_pruned = []
    for l in signup:
        if len(l.books) > 0:
            signup_pruned.append(ScanTaskSignUp(l.id, len(l.books), l.books))
    return signup_pruned

Compute solution here. Solution runner only executes `compute`.

In [10]:
def compute_baseline(need):
    return [ScanTaskSignUp(0, 1, [0])]

def lib_int(l):
    # l.max_output * l.sum_scores / (l.signup_days*100) not better
    # l.max_output * l.sum_scores * len(l.unique_books) / (l.signup_days) not better
    # l.max_output * l.sum_scores was good to some extent
    # l.max_output * l.sum_scores * len(l.unique_books) not good 
    # (l.max_output + l.sum_scores) / (l.signup_days) improves f 
    # (0.999*l.max_output + 0.001*l.sum_scores) / (10 * l.signup_days)
    # l.sum_unique_scores
    return (l.max_output + l.sum_scores) / (l.signup_days)

def compute_by_interestingness(scanning, prune=False):
    go = []
    sorted_libs = sorted(scanning.libraries, key=lib_int, reverse=True)
    for l in sorted_libs:
        sorted_books = sort_books_by_scores_desc(scanning.scores, l.books)
        go.append(ScanTaskSignUp(l.id, l.num_books, sorted_books))
    if prune:
        go = prune_dups(scanning, go)
    return go

def compute_by_libs_by_output(scanning, prune=False):
    go = []
    sorted_libs = sort_enhlibraries_by_field(scanning.libraries, 'max_output')
    for l in sorted_libs:
        sorted_books = sort_books_by_scores_desc(scanning.scores, l.books)
        go.append(ScanTaskSignUp(l.id, l.num_books, sorted_books))
    if prune:
        go = prune_dups(scanning, go)
    return go
    
def compute_minsignuptime_sort_books(scanning, prune=False):
    sorted_libs = sort_libraries_by_field(scanning.libraries, reverse=False)
    go = []
    for l in sorted_libs:
        sorted_books = sort_books_by_scores_desc(scanning.scores, l.books)
        go.append(ScanTaskSignUp(l.id, l.num_books, sorted_books))
    if prune:
        go = prune_dups(scanning, go)
    return go

def compute_minsignuptime(scanning):
    sorted_libs = sort_libraries_by_field(scanning.libraries, reverse=False)
    go = []
    for l in sorted_libs:
        go.append(ScanTaskSignUp(l.id, l.num_books, l.books))
    go = prune_dups(scanning, go)
    return go

def compute(scanning):
    return compute_by_interestingness(scanning)

Run, score and store.

In [11]:
s = new_sol()

['a_example.txt', 'b_read_on.txt', 'c_incunabula.txt', 'd_tough_choices.txt', 'e_so_many_books.txt', 'f_libraries_of_the_world.txt']
a: 		21
f: 		5,120,686
c: 		4,800,206
e: 		2,713,843
b: 		5,822,900
d: 		4,815,395
Total: 		23,273,051


---

Example

In [12]:
scanning = parse_input('data/a_example.txt')

In [13]:
s = compute(scanning)

In [14]:
s

[ScanTaskSignUp(id=0, num_books=5, books=[3, 4, 2, 1, 0]),
 ScanTaskSignUp(id=1, num_books=4, books=[5, 3, 2, 0])]

In [15]:
prune_dups(scanning, s)

[ScanTaskSignUp(id=0, num_books=5, books=[3, 4, 2, 1, 0]),
 ScanTaskSignUp(id=1, num_books=1, books=[5])]